In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp
import nashpy as nash
import pulp
from itertools import combinations

## Set-Up:

- Dataframe Loading
- Base Tradeoff Matrix
- Team - Vector Conversion Functions
- Example Teams

In [22]:
#Global Variables, don't change these, unless you are me:

payoff_df = pd.read_csv("MarvelRivals_Payoff_Matrix.csv")
hero_list = hero_list = list(payoff_df.iloc[:, 0])

payoff_df = pd.read_csv("MarvelRivals_Payoff_Matrix.csv",  index_col=0)
payoff_matrix = payoff_df.values

def team_to_vector(selected_heroes):
    vector = np.zeros(len(hero_list), dtype=int)
    for i, hero in enumerate(hero_list):
        if hero in selected_heroes:
            vector[i] = 1
    return vector
    
def vector_to_team(binary_vector):
    selected_heroes = [hero_list[i] for i in range(len(hero_list)) if binary_vector[i] == 1]
    return selected_heroes

Avengers = ["Black Widow", "Iron Man", "Thor", "Hulk", "Hawkeye", "Captain America"]
FantasticFour = ["Invisible Woman", "The Thing", "Human Torch", "Mister Fantastic", "Spider Man", "Wolverine"]
Guardians = ["Mantis", "Rocket Raccoon", "Adam Warlock", "Star Lord", "Groot", "Venom"]
Xmen = ["Magneto", "Psylocke", "Wolverine", "Magik", "Storm", "Namor"]
RandomTeam = ["Magneto", "Doctor Strange", "The Punisher", "Scarlet Witch", "Cloak & Dagger", "Loki"]

bestTeam = ['Captain America', 'Black Panther', 'Magik', 'Storm', 'Mantis', 'Rocket Raccoon']

In [23]:
MyTeam = ['Doctor Strange', 'Groot', 'The Punisher', 'Winter Soldier', "Cloak & Dagger", "Invisible Woman"]
OpponentTeam = ["Doctor Strange", "Peni Parker", "Magik", "Moon Knight", "Cloak & Dagger", "Invisible Woman"]

rowPlayer = team_to_vector(Guardians)
columnPlayer = team_to_vector(Guardians)

result = rowPlayer.T @ payoff_matrix @ columnPlayer
result

np.float64(0.0)

## Base Question Implementation: Optimal Conter Strategy given known opponent Strategy

It turns out, given an opponent team, it is relatively easy to figure out the optimal conter team using cvxpy.

In [24]:
# In order to use, first use team_to_vector() to convert the team into a binary vector
def base_optimal_conter_info(opponent_team):
    x = cp.Variable(37, boolean=True)
    y = opponent_team
    constraint_0 = [cp.sum(x) == 6]
    constraint_1 = [cp.sum(y) == 6]
    
    obj = cp.Maximize(x.T @ payoff_matrix @ y)
    prob = cp.Problem(obj,constraint_0+constraint_1)
    prob.solve()

    optimal_team = vector_to_team(x.value.round())
    expected_value = x.value.T @ payoff_matrix @ y

    print("Optimal Team:")
    for hero in optimal_team:
        print(f"-{hero}")
    print(f"Expected Utility Score of (x^T * A * y) is: {expected_value:.2f}")
    return x.value

def base_optimal_conter(opponent_team):
    x = cp.Variable(37, boolean=True)
    y = opponent_team
    constraint_0 = [cp.sum(x) == 6]
    constraint_1 = [cp.sum(y) == 6]
    
    obj = cp.Maximize(x.T @ payoff_matrix @ y)
    prob = cp.Problem(obj,constraint_0+constraint_1)
    prob.solve()

    return x.value

In [28]:
# Example Usage:
OpponentTeam = ["Doctor Strange", "Peni Parker", "Magik", "Moon Knight", "Cloak & Dagger", "Invisible Woman"]
y = team_to_vector(bestTeam)
print("Best Conter to OpponentTeam:", ', '.join(bestTeam), "is:")
OpponentTeamConter = base_optimal_conter_info(y)

Best Conter to OpponentTeam: Captain America, Black Panther, Magik, Storm, Mantis, Rocket Raccoon is:
Optimal Team:
-Captain America
-Iron Fist
-Magik
-Storm
-Mantis
-Rocket Raccoon
Expected Utility Score of (x^T * A * y) is: 0.35


## Secondary Question: Optimal Strategies for the payoff matrix

Since we have defined the row and column players and it clearly is a zero sum game, we can figure out he equilbrium state and its cooresponding optimal strategy of it, hence the "best team" highlightened in von neumann minimax theorem

In [ ]:
# 37 choose 6 is not very big so use Brutal Force Method:
def row_player_maximin_binary(payoff_matrix):
    n = payoff_matrix.shape[0]
    assert payoff_matrix.shape == (37, 37), "Expected a 37x37 payoff matrix."
    
    best_value = float('-inf')
    best_vector = None
    
    for row_subset in combinations(range(n), 6):
        c = np.sum(payoff_matrix[list(row_subset), :], axis=0)
        
        chosen_cols = np.partition(c, 6)[:6]
        payoff = np.sum(chosen_cols)
        
        if payoff > best_value:
            best_value = payoff
            vector = np.zeros(n, dtype=int)
            vector[list(row_subset)] = 1
            best_vector = vector

    print(f"Row-player guaranteed payoff is greater than: {best_value:.2f}")
    BestTeam = vector_to_team(vector)
    print("Von Neumann MinMax Best team is:", ', '.join(BestTeam))
    
    return best_vector

In [27]:
OptimalVector = row_player_maximin_binary(payoff_matrix)
print("Best Conter to von Neumann optimal team is:")
BestTeamConter = base_optimal_conter_info(OptimalVector)

Row-player guaranteed payoff is greater than: -0.12
Von Neumann MinMax Best team is: Captain America, Iron Fist, Magik, Storm, Mantis, Rocket Raccoon
Best Conter to von Neumann optimal team is:
Optimal Team:
-Captain America
-Peni Parker
-Magik
-Storm
-Mantis
-Rocket Raccoon
Expected Utility Score of (x^T * A * y) is: 0.12


## Secondary Question: Character Ban Stragegy Implementation

In the game, once you hit a certain rank (by being good at the game), the game introduce a new mechanism where a team can collectively ban two characters where the opponent team can no longer choose, this makes the character counter strategy more interesting.